# **📌 AI-BASED Personalized Learning System – Project Report Notes**

                                              📄 DECLARATION

I hereby declare that this project titled “AI-Based Personalized Learning System for Adaptive Content Recommendation” is my original work and has not been submitted elsewhere.
  
  

> Name : **Kuldeep Pandey**  
> Studend Code : **iitrpr_ai_25010916**   
> Course : **Minor In Artificial Intelligence ( AI )**  
> Objective : **Module E Project - AI in Personalised Learning**    
> Date : **24/11/2025**

                                              📘 ABSTRACT

This project implements an AI-powered Personalized Learning System that adapts study material, question difficulty, and topic recommendations based on each student's performance.

The system analyzes factors such as accuracy, speed, difficulty score, and topic-wise behavior to predict performance and generate personalized pathways.
A Random Forest classifier is used to predict the correctness of future responses, while rule-based logic recommends adaptive difficulty levels.

The project includes dataset preparation, preprocessing, feature engineering, model training, evaluation, and personalized insights.
The trained model can recommend suitable study paths and help improve learning outcomes.

                                          📍 TABLE OF CONTENTS

1-) Problem Definition & Objective   
2-) Data Understanding & Preparation   
3-) Model / System Design    
4-) Core Implementation  
5-) Evaluation & Analysis     
6-) Ethical Considerations & Responsible AI       
7-) Conclusion & Future Scope      


                                              # 1. Problem Definition & Objective

## a-) Selected Project Track :
**AI + Education (Personalized Learning System / Recommendation System / Adaptive Learning)**

## b-) Clear Problem Statement :
Traditional learning systems follow a one-size-fits-all approach. However, learners have different learning speeds, engagement levels, strengths, and weaknesses.  
This project builds an AI-powered personalized learning system that adapts:
>- quiz difficulty  
>- learning content format  
>- hints and feedback strategy  
>- personalized recommendations  

based on learner performance and engagement history.

## c-) Real-World Relevance and Motivation :
Personalized learning is used in real-world platforms like:
>- Duolingo (adaptive practice)
>- Khan Academy (skill mastery)
>- Coursera / EdTech platforms (recommendations)

This system helps learners:
>- improve weak areas
>- get suitable difficulty questions
>- stay engaged using adaptive feedback and analytics  


                                             # 2. Data Understanding & Preparation

## a-) Dataset Source :
This notebook supports:
>- **Sample synthetic dataset (default)**
>- **Random 500-row learner dataset generation**
>- **Custom dataset upload** (CSV/JSON learner profiles, quiz history, question bank)

## b-) Data Loading and Exploration :
We load learner profiles and quiz history into structured Python dictionaries & DataFrames.

## c-) Cleaning, Preprocessing, Feature Engineering :
We extract key features like:
>- average accuracy
>- average time per quiz
>- hints used
>- retries
>- topic-wise performance

## d-) Handling Missing Values or Noise :
We handle missing values using safe defaults:
>- engagement_score = 70
>- learning_pace = "moderate"
>- classification = "Average Learner"

                                              # 3. Model / System Design

## a-) AI Technique Used :
This project uses a **Hybrid AI approach**:
>- Rule-based learner classification
>- Adaptive difficulty prediction
>- K-Means clustering for learner grouping
>- Recommendation engine based on learner category

## b-) Architecture / Pipeline :
>1. Load learner data + quiz history  
2. Extract performance metrics  
3. Classify learner: Struggling / Average / Advanced  
4. Predict next quiz difficulty  
5. Generate recommendations  
6. Cluster learners using K-Means  
7. Show analytics and sample outputs  

## c-) Justification of Design Choices :
>- Rule-based classification ensures interpretability  
>- KMeans helps identify similar learner patterns  
>- Adaptive difficulty improves motivation and reduces frustration  
>- Recommendations provide personalized next steps  

                                              # 4. Core Implementation

This section demonstrates the end-to-end pipeline execution:
>- Update learner classifications  
>- Generate adaptive settings  
>- Produce recommendations  
>- Cluster learners  

                                              # 5. Evaluation & Analysis

## a-) Metrics Used :
>- Average Accuracy (%)
>- Average Time per Quiz (seconds)
>- Hints Used
>- Adaptive Difficulty prediction quality (qualitative)

## b-) Sample Outputs :
We generate:
>- learner classification
>- recommended next difficulty
>- personalized recommendations
>- learner cluster id

## c-) Performance Analysis & Limitations :
Limitations:
>- Rule-based classification is simple and may not generalize perfectly  
>- KMeans clustering depends on dataset size and scaling  
>- No deep learning model is used (lightweight system design)  

                                              # 6. Ethical Considerations & Responsible AI

## a) Bias and Fairness Considerations
- Engagement score and quiz performance may be influenced by external factors  
  (internet access, language barriers, stress, disabilities).  
- The system must not label learners permanently.  
- Recommendations should motivate learners rather than discourage them.  

## b) Dataset Limitations
- Synthetic datasets may not represent real classroom diversity.  
- Quiz history may be incomplete or noisy.  

## c) Responsible Use of AI Tools
- AI should support teachers and learners, not replace educators.  
- Learner data must be stored securely.  
- Personal data should not be exposed in public dashboards.

                                              # 7. Conclusion & Future Scope

## a) Summary of Results
This notebook demonstrated a working AI-powered personalized learning system that:
- classifies learners into categories  
- adapts difficulty dynamically  
- generates recommendations  
- clusters learners based on performance patterns  

## b) Possible Improvements and Extensions
Future work can include:
- Deep learning based learner modeling  
- LLM-based tutor responses with prompt engineering  
- Reinforcement learning for difficulty adaptation  
- Real-time dashboards + gamification badges  
- Deployment using Streamlit + database integration  



In [1]:
import pandas as pd
import numpy as np
import json
import random
from datetime import datetime
from collections import defaultdict

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [2]:
# -------------------------
# Sample Learner Dataset
# -------------------------
def generate_sample_learners():
    learners = {
        "Alice Johnson": {
            "id": "L001",
            "quiz_history": [
                {"accuracy": 45, "avg_time": 85, "hints_used": 8, "retries": 3, "date": "2026-01-10", "topic": "Algebra"},
                {"accuracy": 52, "avg_time": 78, "hints_used": 6, "retries": 2, "date": "2026-01-12", "topic": "Geometry"},
                {"accuracy": 48, "avg_time": 82, "hints_used": 7, "retries": 3, "date": "2026-01-14", "topic": "Algebra"}
            ],
            "engagement_score": 62,
            "learning_pace": "slow",
            "classification": "Struggling Learner",
            "strengths": ["Visual Learning", "Pattern Recognition"],
            "weaknesses": ["Abstract Reasoning", "Time Management"]
        },
        "Bob Smith": {
            "id": "L002",
            "quiz_history": [
                {"accuracy": 75, "avg_time": 45, "hints_used": 2, "retries": 0, "date": "2026-01-10", "topic": "Algebra"},
                {"accuracy": 78, "avg_time": 42, "hints_used": 1, "retries": 0, "date": "2026-01-12", "topic": "Geometry"},
                {"accuracy": 73, "avg_time": 48, "hints_used": 2, "retries": 1, "date": "2026-01-14", "topic": "Statistics"}
            ],
            "engagement_score": 78,
            "learning_pace": "moderate",
            "classification": "Average Learner",
            "strengths": ["Consistent Performance", "Good Understanding"],
            "weaknesses": ["Complex Problem Solving", "Advanced Concepts"]
        },
        "Carol Williams": {
            "id": "L003",
            "quiz_history": [
                {"accuracy": 92, "avg_time": 28, "hints_used": 0, "retries": 0, "date": "2026-01-10", "topic": "Algebra"},
                {"accuracy": 95, "avg_time": 25, "hints_used": 0, "retries": 0, "date": "2026-01-12", "topic": "Geometry"},
                {"accuracy": 94, "avg_time": 26, "hints_used": 0, "retries": 0, "date": "2026-01-14", "topic": "Calculus"}
            ],
            "engagement_score": 95,
            "learning_pace": "fast",
            "classification": "Advanced Learner",
            "strengths": ["Quick Comprehension", "Problem Solving", "Self-Directed"],
            "weaknesses": ["May skip fundamentals", "Needs challenges"]
        }
    }
    return learners

learners_data_sample = generate_sample_learners()
list(learners_data_sample.keys())


['Alice Johnson', 'Bob Smith', 'Carol Williams']

In [3]:
# -------------------------
# Random 500-row Dataset Generator
# -------------------------
first_names = ["Aarav","Vivaan","Aditya","Vihaan","Arjun","Sai","Krishna","Ishaan","Rahul","Rohan","Ananya","Aditi","Ira","Saanvi","Diya","Meera","Priya","Nisha","Kavya","Neha"]
last_names = ["Sharma","Verma","Singh","Gupta","Mehta","Kumar","Yadav","Patel","Jain","Bansal","Chopra","Kapoor","Reddy","Iyer","Nair","Das","Roy","Malhotra","Saxena","Mishra"]

paces = ["slow","moderate","fast"]
classes = ["Struggling Learner","Average Learner","Advanced Learner"]

strength_pool = ["Visual Learning","Pattern Recognition","Problem Solving","Quick Comprehension","Self-Directed","Consistency","Logical Thinking","Persistence","Good Understanding"]
weakness_pool = ["Abstract Reasoning","Time Management","Complex Problems","Confidence","Speed","Advanced Concepts","Careless Mistakes","Focus"]

def generate_random_learners(n=500):
    data = []
    for i in range(1, n+1):
        name = f"{random.choice(first_names)} {random.choice(last_names)}"
        learner_id = f"L{i:03d}"
        engagement = random.randint(40, 100)
        pace = random.choice(paces)
        classification = random.choice(classes)

        strengths = random.sample(strength_pool, k=random.randint(1,3))
        weaknesses = random.sample(weakness_pool, k=random.randint(1,3))

        data.append({
            "learner_name": name,
            "learner_id": learner_id,
            "engagement_score": engagement,
            "learning_pace": pace,
            "classification": classification,
            "strengths": ",".join(strengths),
            "weaknesses": ",".join(weaknesses)
        })
    return pd.DataFrame(data)

df_random_500 = generate_random_learners(500)
df_random_500.head()


,learner_name,learner_id,engagement_score,learning_pace,classification,strengths,weaknesses
0,Ananya Iyer,L001,85,moderate,Advanced Learner,Logical Thinking,Confidence
1,Nisha Das,L002,63,moderate,Average Learner,"Logical Thinking,Quick Comprehension","Speed,Advanced Concepts,Complex Problems"
2,Krishna Malhotra,L003,89,moderate,Struggling Learner,"Pattern Recognition,Logical Thinking,Good Unde...","Complex Problems,Careless Mistakes"
3,Vivaan Singh,L004,55,fast,Struggling Learner,"Problem Solving,Quick Comprehension","Speed,Advanced Concepts"
4,Diya Kapoor,L005,44,moderate,Struggling Learner,"Pattern Recognition,Problem Solving,Consistency",Focus


In [4]:
# Convert random dataset into the same dictionary structure used by the system
def random_df_to_learners_dict(df):
    learners = {}
    for _, row in df.iterrows():
        learners[row["learner_name"]] = {
            "id": row["learner_id"],
            "quiz_history": [],
            "engagement_score": int(row["engagement_score"]),
            "learning_pace": row["learning_pace"],
            "classification": row["classification"],
            "strengths": row["strengths"].split(","),
            "weaknesses": row["weaknesses"].split(",")
        }
    return learners

learners_data_random = random_df_to_learners_dict(df_random_500)
len(learners_data_random)


286

In [5]:
# Exploration helper: learners dict -> dataframe
def learners_to_dataframe(learners_data):
    rows = []
    for name, data in learners_data.items():
        quiz_history = data["quiz_history"]
        avg_acc = np.mean([q["accuracy"] for q in quiz_history]) if quiz_history else np.nan
        avg_time = np.mean([q["avg_time"] for q in quiz_history]) if quiz_history else np.nan

        rows.append({
            "learner_name": name,
            "learner_id": data["id"],
            "engagement_score": data["engagement_score"],
            "learning_pace": data["learning_pace"],
            "classification": data["classification"],
            "avg_accuracy": avg_acc,
            "avg_time": avg_time,
            "strengths": ", ".join(data["strengths"]),
            "weaknesses": ", ".join(data["weaknesses"]),
            "quizzes_completed": len(quiz_history)
        })
    return pd.DataFrame(rows)

df_sample = learners_to_dataframe(learners_data_sample)
df_sample


,learner_name,learner_id,engagement_score,learning_pace,classification,avg_accuracy,avg_time,strengths,weaknesses,quizzes_completed
0,Alice Johnson,L001,62,slow,Struggling Learner,48.333333,81.666667,"Visual Learning, Pattern Recognition","Abstract Reasoning, Time Management",3
1,Bob Smith,L002,78,moderate,Average Learner,75.333333,45.000000,"Consistent Performance, Good Understanding","Complex Problem Solving, Advanced Concepts",3
2,Carol Williams,L003,95,fast,Advanced Learner,93.666667,26.333333,"Quick Comprehension, Problem Solving, Self-Dir...","May skip fundamentals, Needs challenges",3


In [6]:
# -------------------------
# Learner Classification
# -------------------------
def classify_learner(learner_data):
    quiz_history = learner_data["quiz_history"]
    if not quiz_history:
        return "New Learner"

    avg_accuracy = np.mean([q["accuracy"] for q in quiz_history])
    avg_time = np.mean([q["avg_time"] for q in quiz_history])
    total_hints = sum([q["hints_used"] for q in quiz_history])

    if avg_accuracy < 60 and (avg_time > 70 or total_hints > 5):
        return "Struggling Learner"
    elif avg_accuracy >= 85 and avg_time < 35 and total_hints == 0:
        return "Advanced Learner"
    else:
        return "Average Learner"


In [7]:
# -------------------------
# Difficulty Prediction
# -------------------------
def predict_next_difficulty(learner_data):
    quiz_history = learner_data["quiz_history"]
    if not quiz_history:
        return "easy"

    recent = quiz_history[-3:] if len(quiz_history) >= 3 else quiz_history
    avg_accuracy = np.mean([q["accuracy"] for q in recent])
    avg_time = np.mean([q["avg_time"] for q in recent])

    if avg_accuracy >= 90 and avg_time < 35:
        return "hard"
    elif avg_accuracy >= 70 and avg_time < 60:
        return "medium"
    else:
        return "easy"


In [8]:
# -------------------------
# Adaptation Engine
# -------------------------
def adapt_content(learner_data):
    classification = learner_data["classification"]
    quiz_history = learner_data["quiz_history"]

    adaptations = {
        "difficulty": predict_next_difficulty(learner_data),
        "content_format": "visual" if classification == "Struggling Learner" else "text",
        "enable_challenge_mode": classification == "Advanced Learner",
        "provide_hints": classification != "Advanced Learner",
        "revision_needed": False
    }

    if quiz_history:
        recent_accuracy = quiz_history[-1]["accuracy"]
        if recent_accuracy < 50:
            adaptations["revision_needed"] = True
            adaptations["difficulty"] = "easy"

    return adaptations


In [9]:
# -------------------------
# Recommendation Engine
# -------------------------
def generate_recommendations(learner_data, current_topic="Algebra"):
    classification = learner_data["classification"]

    if classification == "Struggling Learner":
        return [
            {"type": "📹 Video Tutorial", "title": f"{current_topic} Basics", "priority": "High"},
            {"type": "✏️ Guided Practice", "title": "Practice with Hints", "priority": "High"},
            {"type": "📝 Revision Notes", "title": f"{current_topic} Quick Revision", "priority": "Medium"},
        ]
    elif classification == "Advanced Learner":
        return [
            {"type": "🏆 Challenge Problems", "title": f"Advanced {current_topic}", "priority": "High"},
            {"type": "🔬 Mini Project", "title": "Real-World Application Task", "priority": "Medium"},
            {"type": "📚 Next Topic", "title": f"Beyond {current_topic}", "priority": "High"},
        ]
    else:
        return [
            {"type": "📝 Mixed Practice", "title": f"{current_topic} Practice Set", "priority": "High"},
            {"type": "🎯 Skill Builder", "title": "Target Weak Areas", "priority": "High"},
            {"type": "⚡ Daily Quiz", "title": "Daily Challenge", "priority": "Medium"},
        ]


In [10]:
# -------------------------
# KMeans Clustering
# -------------------------
def cluster_learners(learners_data):
    features = []
    names = []

    for name, data in learners_data.items():
        if data["quiz_history"]:
            avg_accuracy = np.mean([q["accuracy"] for q in data["quiz_history"]])
            avg_time = np.mean([q["avg_time"] for q in data["quiz_history"]])
            features.append([avg_accuracy, avg_time])
            names.append(name)

    if len(features) < 3:
        return {}

    scaler = StandardScaler()
    X = scaler.fit_transform(features)

    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X)

    return {n: int(l) for n, l in zip(names, labels)}


In [11]:
# Apply classification updates on sample dataset
for learner_name in learners_data_sample:
    learners_data_sample[learner_name]["classification"] = classify_learner(learners_data_sample[learner_name])

df_updated = learners_to_dataframe(learners_data_sample)
df_updated


,learner_name,learner_id,engagement_score,learning_pace,classification,avg_accuracy,avg_time,strengths,weaknesses,quizzes_completed
0,Alice Johnson,L001,62,slow,Struggling Learner,48.333333,81.666667,"Visual Learning, Pattern Recognition","Abstract Reasoning, Time Management",3
1,Bob Smith,L002,78,moderate,Average Learner,75.333333,45.000000,"Consistent Performance, Good Understanding","Complex Problem Solving, Advanced Concepts",3
2,Carol Williams,L003,95,fast,Advanced Learner,93.666667,26.333333,"Quick Comprehension, Problem Solving, Self-Dir...","May skip fundamentals, Needs challenges",3


In [12]:
clusters = cluster_learners(learners_data_sample)

results = []
for name, data in learners_data_sample.items():
    adaptations = adapt_content(data)
    recs = generate_recommendations(data, current_topic="Algebra")

    results.append({
        "learner": name,
        "classification": data["classification"],
        "recommended_difficulty": adaptations["difficulty"],
        "challenge_mode": adaptations["enable_challenge_mode"],
        "revision_needed": adaptations["revision_needed"],
        "cluster_id": clusters.get(name, None),
        "top_recommendation": recs[0]["title"]
    })

pd.DataFrame(results)


,learner,classification,recommended_difficulty,challenge_mode,revision_needed,cluster_id,top_recommendation
0,Alice Johnson,Struggling Learner,easy,False,True,1,Algebra Basics
1,Bob Smith,Average Learner,medium,False,False,0,Algebra Practice Set
2,Carol Williams,Advanced Learner,hard,True,False,2,Advanced Algebra
